In [5]:
import pandas as pd
import re

In [11]:
def process_proteome_data(input_file_path, output_file_path):
  """
  Reads the raw proteome data, splits the BUSCO and Taxonomic lineage columns,
  and saves the processed dataframe to a new CSV file.
  """
  
  # Load the dataset
  # We use 'error_bad_lines=False' or 'on_bad_lines' to handle potential parsing issues 
  # depending on pandas version, but standard read_csv usually works for well-formed CSVs.
  try:
    df = pd.read_excel(input_file_path)
  except Exception as e:
    return f"Error reading file: {e}"

    # --- 1. Process BUSCO Column ---
    
  def parse_busco(busco_str):
    """
    Parses BUSCO string like: C:99.8%[S:99.6%,D:0.2%],F:0.0%,M:0.2%,n:904
    Returns a dictionary of values.
    """
        
    if pd.isna(busco_str):
      return {
        'Complete': None,
        'Single': None,
        'Double': None,
        'Fragmented': None,
        'Missing': None,
        'Count': None
      }
        
    # Regex to extract numbers for C, S, D, F, M, n
    # This handles the percentage signs and brackets
    c_match = re.search(r'C:([\d\.]+)%', busco_str)
    s_match = re.search(r'S:([\d\.]+)%', busco_str)
    d_match = re.search(r'D:([\d\.]+)%', busco_str)
    f_match = re.search(r'F:([\d\.]+)%', busco_str)
    m_match = re.search(r'M:([\d\.]+)%', busco_str)
    n_match = re.search(r'n:(\d+)', busco_str)

    return {
      'Complete': float(c_match.group(1)) if c_match else None,
      'Single': float(s_match.group(1)) if s_match else None,
      'Double': float(d_match.group(1)) if d_match else None,
      'Fragmented': float(f_match.group(1)) if f_match else None,
      'Missing': float(m_match.group(1)) if m_match else None,
      'Count': int(n_match.group(1)) if n_match else None
    }

  # Apply the parsing function
  busco_data = df['BUSCO'].apply(parse_busco).apply(pd.Series)
    
  # Concatenate the new columns to the original dataframe
  df = pd.concat([df, busco_data], axis=1)

  # --- 2. Process Taxonomic Lineage Column ---
    
  # Split the lineage string by comma
  # The `expand=True` argument turns the split lists into separate columns
  lineage_split = df['Taxonomic lineage'].str.split(', ', expand=True)
    
  # Assign names to the lineage columns based on standard ranks found in the file
  # Note: The depth of lineage varies. We will name them generically 
  # (Kingdom, Phylum, Class, Order, Family, Genus, Species) based on the typical order 
  # seen in your file: "cellular organisms, Archaea, Methanobacteriati, Methanobacteriota, Stenosarchaea group, Halobacteria, Halobacteriales..."
  
  # Depending on the strictness of the taxonomy in the file, we might have variable column counts.
  # We will name the first 7 columns which seem consistent in the snippet.
  new_column_names = [
    'Taxonomy_Root',         # cellular organisms
    'Taxonomy_Domain',       # Archaea
    'Taxonomy_Clade1',       # Methanobacteriati (varies)
    'Taxonomy_Phylum',       # Methanobacteriota
    'Taxonomy_Clade2',       # Stenosarchaea group
    'Taxonomy_Class',        # Halobacteria
    'Taxonomy_Order',        # Halobacteriales
    'Taxonomy_Family',       # e.g. Haloarculaceae
    'Taxonomy_Genus',        # e.g. Haloarcula
    'Taxonomy_Species'       # e.g. Haloarcula rubripromontorii (if present)
  ]
    
  # Handle cases where the split resulted in more or fewer columns than our names list
  current_cols = lineage_split.shape[1]
    
  # Generate generic names for any extra columns beyond our named list
  if current_cols > len(new_column_names):
    extra_cols = [f'Taxonomy_Level_{i+1}' for i in range(len(new_column_names), current_cols)]
    final_col_names = new_column_names + extra_cols
  else:
    final_col_names = new_column_names[:current_cols]
        
  lineage_split.columns = final_col_names
    
  # Concatenate the new lineage columns
  df = pd.concat([df, lineage_split], axis=1)

  # --- 3. Cleanup ---
    
  # Optionally drop the original raw columns if you don't need them anymore
  df.drop(columns=['BUSCO', 'Taxonomic lineage'], inplace=True)

  # Print the length of the dataset
  print(f"Dataset length: {len(df)}")

  # Save to new file
  df.to_csv(output_file_path, index=False)
    
  return df.head()

# Example usage (Replace filenames with your actual file paths)
input_csv = 'proteomes_Halobacteriales_2025_12_04.xlsx'
output_csv = 'processed_proteomes_data.csv'

# Run the function
df_head = process_proteome_data(input_csv, output_csv)
print("Processing complete. First 5 rows of processed data:")
df_head

Dataset length: 843
Processing complete. First 5 rows of processed data:


/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Proteome Id,Organism,Organism Id,Protein count,CPD,Taxon mnemonic,Components,Genome assembly ID,Genome representation,Complete,...,Taxonomy_Domain,Taxonomy_Clade1,Taxonomy_Phylum,Taxonomy_Clade2,Taxonomy_Class,Taxonomy_Order,Taxonomy_Family,Taxonomy_Genus,Taxonomy_Species,Taxonomy_Level_11
0,UP000037729,Haloarcula rubripromontorii,1705562,3788,Unknown,9EURY,Unassembled WGS sequence,GCA_001280425.1,full,99.8,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Haloarculaceae,Haloarcula,None,None
1,UP000037747,Halorubrum tropicale,1765655,3245,Unknown,9EURY,Unassembled WGS sequence,GCA_001280455.1,full,99.7,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Haloferacaceae,Halorubrum,None,None
2,UP000050535,Halolamina pelagica,699431,3464,Unknown,9EURY,Unassembled WGS sequence,GCA_001307315.1,full,77.8,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Haloferacaceae,Halolamina,None,None
3,UP000066737,Halobacterium hubeiense,1407499,3340,Unknown,9EURY,Chromosome I; Plasmid pSTJ001; Plasmid pSTJ003...,GCA_001488575.1,full,99.3,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Halobacteriaceae,Halobacterium,None,None
4,UP000069906,Halanaeroarchaeum sulfurireducens,1604004,2225,Unknown,9EURY,Chromosome; Plasmid pHSR2-01,GCA_001011115.1,full,92.7,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Halobacteriaceae,Halanaeroarchaeum,None,None


In [12]:
def complete_and_contamination_filter(file):
  try:
    df = pd.read_csv(file)
  except Exception as e:
    print(f"Unable to read file: {file}")
    return f'Error {e}'

  print(f'Before filter: {len(df)}')
  df = df[(df['Complete'] >= 90.0) & (df['Missing'] <= 5.0)]
  print(f'After filter: {len(df)}')
  df.to_csv(file, index=False)

  return df.head()


df_head = complete_and_contamination_filter(output_csv)
df_head

Before filter: 843
After filter: 523


,Proteome Id,Organism,Organism Id,Protein count,CPD,Taxon mnemonic,Components,Genome assembly ID,Genome representation,Complete,...,Taxonomy_Domain,Taxonomy_Clade1,Taxonomy_Phylum,Taxonomy_Clade2,Taxonomy_Class,Taxonomy_Order,Taxonomy_Family,Taxonomy_Genus,Taxonomy_Species,Taxonomy_Level_11
0,UP000037729,Haloarcula rubripromontorii,1705562,3788,Unknown,9EURY,Unassembled WGS sequence,GCA_001280425.1,full,99.8,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Haloarculaceae,Haloarcula,NaN,NaN
1,UP000037747,Halorubrum tropicale,1765655,3245,Unknown,9EURY,Unassembled WGS sequence,GCA_001280455.1,full,99.7,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Haloferacaceae,Halorubrum,NaN,NaN
3,UP000066737,Halobacterium hubeiense,1407499,3340,Unknown,9EURY,Chromosome I; Plasmid pSTJ001; Plasmid pSTJ003...,GCA_001488575.1,full,99.3,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Halobacteriaceae,Halobacterium,NaN,NaN
6,UP000198775,Halorientalis persicus,1367881,5137,Outlier (high value),9EURY,Unassembled WGS sequence,GCA_900110215.1,full,99.4,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Haloarculaceae,Halorientalis,NaN,NaN
7,UP000198848,Natronobacterium texcoconense,1095778,4046,Outlier (high value),NATTX,Unassembled WGS sequence,GCA_900104065.1,full,98.2,...,Archaea,Methanobacteriati,Methanobacteriota,Stenosarchaea group,Halobacteria,Halobacteriales,Natrialbaceae,Natronobacterium,NaN,NaN
